# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from api_keys import g_key

In [2]:
path = "output_data.csv"

In [3]:
data = pd.read_csv(path)
data.columns

Index(['Unnamed: 0', 'city', 'Lat', 'Lng', 'Max Temp', 'Humidity',
       'Cloudiness', 'Wind Speed', 'Country', 'Date'],
      dtype='object')

In [4]:
del data['Unnamed: 0']

In [5]:
data.head()

,city,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Karratha,-20.7377,116.8463,66.81,67,10,10.27,AU,1627498541
1,Hermanus,-34.4187,19.2345,55.80,71,6,16.26,ZA,1627498654
2,Jacareacanga,-6.2222,-57.7528,98.91,34,20,3.44,BR,1627498654
3,Aksu,41.1231,80.2644,85.98,28,50,8.32,CN,1627498552
4,Iqaluit,63.7506,-68.5145,49.73,61,75,5.01,CA,1627498496


In [6]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)
data= data.dropna()
# Store 'Lat' and 'Lng' into  locations 
locations = data[["Lat", "Lng"]].astype(float)

# Convert Humidity to float and store
# HINT: be sure to handle NaN values
humidity = data["Humidity"].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure()

humidity_layer = gmaps.heatmap_layer(locations, weights= humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(humidity_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
data_1 = data.loc[data['Max Temp']> 70]
data_1 = data_1.loc[data['Max Temp']< 80]
print([min(data_1['Max Temp']), max(data_1['Max Temp'])])
data_1.shape

[70.05, 79.97]


(137, 9)

In [8]:
data_1 = data_1.loc[data['Wind Speed']< 10]
print(max(data_1['Wind Speed']))
data_1.shape

9.84


(98, 9)

In [9]:
data_1 = data_1.loc[data['Cloudiness']== 0]
print(max(data_1['Cloudiness']))
data_1.shape

0


(12, 9)

In [10]:
data_1 = data_1.reset_index(drop = True)

In [11]:
hotel_df = data_1
hotel_df.head()

,city,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Skellefteå,64.7507,20.9528,75.56,41,0,4.61,SE,1627498677
1,Illapel,-31.6308,-71.1653,73.80,10,0,4.88,CL,1627498571
2,Saint-François,46.4154,3.9054,72.84,60,0,3.44,FR,1627498741
3,Muhos,64.8080,25.9931,73.18,71,0,3.22,FI,1627498541
4,Ürümqi,43.8010,87.6005,79.75,32,0,4.47,CN,1627498476


In [12]:
hotel_df['Hotel Name'] = " "
hotel_df

,city,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Skellefteå,64.7507,20.9528,75.56,41,0,4.61,SE,1627498677,
1,Illapel,-31.6308,-71.1653,73.80,10,0,4.88,CL,1627498571,
2,Saint-François,46.4154,3.9054,72.84,60,0,3.44,FR,1627498741,
3,Muhos,64.8080,25.9931,73.18,71,0,3.22,FI,1627498541,
4,Ürümqi,43.8010,87.6005,79.75,32,0,4.47,CN,1627498476,
5,Copiapó,-27.3667,-70.3333,71.87,23,0,6.44,CL,1627498813,
6,Maragogi,-9.0122,-35.2225,77.76,74,0,6.06,BR,1627498815,
7,Vallenar,-28.5708,-70.7581,72.34,20,0,5.26,CL,1627498866,
8,Severnyy,50.6770,36.5532,73.44,64,0,4.47,RU,1627498875,
9,Zharkent,44.1628,80.0000,75.58,34,0,3.02,KZ,1627498881,


In [13]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
   
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.


In [14]:
hotel_df = hotel_df.loc[hotel_df['Hotel Name'] != " "]

In [15]:
hotel_df = hotel_df.reset_index(drop = True)
hotel_df

,city,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Skellefteå,64.7507,20.9528,75.56,41,0,4.61,SE,1627498677,Scandic Skellefteå
1,Illapel,-31.6308,-71.1653,73.80,10,0,4.88,CL,1627498571,Hotel Casablanca
2,Muhos,64.8080,25.9931,73.18,71,0,3.22,FI,1627498541,Montta - active camping / campsite Montan
3,Ürümqi,43.8010,87.6005,79.75,32,0,4.47,CN,1627498476,Sheraton Urumqi Hotel
4,Copiapó,-27.3667,-70.3333,71.87,23,0,6.44,CL,1627498813,Hotel Atacama Suites
5,Maragogi,-9.0122,-35.2225,77.76,74,0,6.06,BR,1627498815,Pousada da Orla
6,Vallenar,-28.5708,-70.7581,72.34,20,0,5.26,CL,1627498866,Hotel Puerto de Vega
7,Severnyy,50.6770,36.5532,73.44,64,0,4.47,RU,1627498875,Hotel VOYAGE
8,Zharkent,44.1628,80.0000,75.58,34,0,3.02,KZ,1627498881,гостиница Satti
9,Jinchang,38.4953,102.1739,75.24,21,0,8.46,CN,1627498938,Jinchuan Hotel


In [16]:
hotel_df.to_csv("hotel.csv")

In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
marker_locations = hotel_df[["Lat", "Lng"]]

In [18]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)
data= data.dropna()
# Store 'Lat' and 'Lng' into  locations 
locations = data[["Lat", "Lng"]].astype(float)

# Convert Humidity to float and store
# HINT: be sure to handle NaN values
humidity = data["Humidity"].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure()

humidity_layer = gmaps.heatmap_layer(locations, weights= humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(humidity_layer)

markers = gmaps.marker_layer(marker_locations, info_box_content= hotel_info)

fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))